## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi </a>  2024/9/10

#### I found that the 'LudRules' column seems to have many features that can be constructed, and here we need to parse the data. I am trying to parse the data here.

In [ ]:
import polars as pl#和pandas类似,但是处理大型数据集有更好的性能.
#necessary
import pandas as pd#导入csv文件的库
import numpy as np #对矩阵进行科学计算的库

train=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
train=train.to_pandas()
print(f"len(train):{len(train)}")
train['LudRules'].head()

In [ ]:
#假设这里的符号匹配符合正常的使用规则,符号匹配函数
def symbol_match(rule):
    stack=[]
    data=[]
    for i in range(len(rule)):
        if rule[i] in ['(','{']:
            stack.append(rule[i])
        elif rule[i] in [')','}']:
            stack=stack[:-1]
        elif (rule[i]=='"') and (len(stack)>0) and (stack[-1]=='"'):
            stack=stack[:-1]
        elif rule[i]=='"':
            stack.append('"')
        data.append(rule[i])
        if len(stack)==0:
            return ''.join(data).strip(),rule[i+1:].strip()#去掉首尾的空格
    #程序走到这里可能是出错了,但是以防万一。
    return '',''
    
def get_ruledata(rule):
    #把字符串前面的'(game '和后面的')'去掉
    rule=rule[len('(game '):-1].strip()#去掉首尾的空格
    datas=[]
    while len(rule):
        data,rule=symbol_match(rule)
        datas.append(data)
    return datas
rule_datas=[]
rules=train['LudRules'].values
for i in range(len(rules)):
    datas=get_ruledata(rules[i])
    rule_datas.append(datas)
rule_datas=np.array(rule_datas,dtype=object)
rule_datas=pd.DataFrame(rule_datas)
rule_datas.columns=['game','players','equipment','rules']
rule_datas.head()